In [61]:
# Load data
import os
import numpy as np
from pydub import AudioSegment
import csv
import scipy


def load_data(path):
    """
    Output structure:
        list of dictionaries
            playlist[n] = {
                name (string)
                audio_array (np.array)
                sampling_rate (double)
                ...
                real_bpm (Int)
            }
    """

    print(f"Loading data from {path}...")

    playlist = []

    for root, dirs, files in os.walk(path, topdown=False):
        for file in files:

            if file == ".DS_Store":
                continue

            audio_file = AudioSegment.from_wav(os.path.join(root, file))

            if audio_file.channels > 1:
                # make sure we are only using one channel. It may not matter.
                audio_file = audio_file.split_to_mono()[0]

            audio_array = np.array(audio_file.get_array_of_samples(), dtype=float)
            song_name, artist_name = extract_names(file)

            song_dict = {
                "artist_name": artist_name,
                "song_name": song_name,
                "audio_segment": audio_file,
                "audio_array": audio_array,
                "song_path": os.path.join(root, file),
            }

            playlist.append(song_dict)

    playlist = basic_feature_extraction(playlist)
    # playlist = load_true_bpm(playlist)

    print(f"\t{len(playlist)} songs loaded")

    return playlist


def extract_names(file):

    song_name, _, artist_name = file.partition(" - ")
    song_name = song_name[3:]

    artist_name, _, _ = artist_name.partition(".")

    return song_name, artist_name


def basic_feature_extraction(playlist):
    """
    Output structure:
        list of dictionaries
            playlist[n] = {
                name (string)
                audio_array (np.array)
                sampling_rate (double)
                ...
            }
    """

    for song in playlist:

        song["frame_rate"] = song["audio_segment"].frame_rate

    return playlist


def load_true_bpm(playlist):

    # load csv with the bpms

    with open("songs.csv", "r") as file:
        csv_reader = csv.DictReader(file, delimiter=",")
        playlist_true_bpm = list(csv_reader)

    for song in playlist:
        flag = 0
        for song_ref in playlist_true_bpm:
            if song["song_name"] == song_ref["song_name"]:
                song["true_bpm"] = song_ref["bpm"]
                flag = 1
        if flag == 0:
            # Don't know if this is the best way of raising an error.
            # Please change to a better one if you know one.
            print("No true bpm found for song:", song["song_name"])

    return playlist


def store_song(mix, path):

    scipy.io.wavfile.write(
        path, rate=mix["frame_rate"], data=mix["audio_array"].astype("int32")
    )


In [115]:
# Relevant feature extraction
# Beat detection
# Key detection
# Structural segmentation

# from librosa.util.utils import frame
import numpy as np
import scipy
import sklearn

from madmom.features.beats import RNNBeatProcessor
from madmom.features.beats import DBNBeatTrackingProcessor
from madmom.features.key import CNNKeyRecognitionProcessor
from madmom.features.key import key_prediction_to_label

import librosa

import essentia
from essentia.standard import FrameGenerator, PeakDetection

import utils


def feature_extraction(playlist):
    print('Extracting features')

    for i, song in enumerate(playlist):
        print(f'\tSong {i+1} / {len(playlist)}')

        print('\t\tEstimating beat...')
        beats_frames, bpm = beat_detection(song)
        song['beat_times'] = beats_frames   # Array like the samples marking with the beat ocurrs, ones/zeros
        song['estimated_bpm'] = bpm         # Int

        print('\t\tEstimating key...')
        key_probabilities, key_label = key_detection(song)
        song['estimated_key'] = key_label.split(' ')[0]        # Probalby string or a int encoding of all the keys
        song['estimated_mode'] = key_label.split(' ')[1]
        song['key_probabilities'] = key_probabilities

        print('\t\tEstimating cue-points')
        cue_points = structural_segmentation(song)
        song['cue_points'] = cue_points      # Array like the samples marking with the cue-point ocurrs 

        # Maybe cut silences or if the cue-points in
        # the beginning and the end are too extreme

    return playlist


# FEATURES

def beat_detection(song):

    proc = DBNBeatTrackingProcessor(fps=100)
    act = RNNBeatProcessor()(song["song_path"])
    beat_times = proc(act)

    # create the array of ones and zeros
    beat_frames = convert_to_frames(beat_times,song)

    # compute the bpm of the song
    bpm = beats_per_minute(beat_times,song)

    return beat_frames, bpm


def convert_to_frames(beat_times, song):

    beat_frames = (beat_times*song["frame_rate"]).astype(int)
    beat_frames_mapped = np.zeros_like(song["audio_array"])
    beat_frames_mapped[beat_frames] = 1
    
    return beat_frames_mapped

def beats_per_minute(beat_times, song):
    
    song_length = len(song["audio_array"])/song["frame_rate"]/60
    beats_count = len(beat_times)
    
    bpm = beats_count/song_length # We could have problems with the first and the last beat
    
    return bpm


def key_detection(song):

    #key = rubberband/madmom (experiment with both)
    
    proc = CNNKeyRecognitionProcessor()
    key_probabilities = proc(song["song_path"])
    key_label = key_prediction_to_label(key_probabilities)

    return key_probabilities, key_label


def structural_segmentation(song):

    kernel_dim = 32
    
    samples_per_beat = int(1.0/(song['estimated_bpm']/(60.0 * song['frame_rate'])))

    frame_size = int(0.5 * samples_per_beat)
    hop_size = int(0.25 * samples_per_beat)

    mfcc_ssm = mfcc_structural_similarity_matrix(song, frame_size=frame_size, hop_size=hop_size)
    rms_ssm = rms_structural_similarity_matrix(song, frame_size=frame_size, hop_size=hop_size)

    kernel = get_checkboard_kernel(kernel_dim)
    mfcc_novelty = apply_kernel(mfcc_ssm, kernel)
    rms_novelty = apply_kernel(rms_ssm, kernel)

    size_dif = mfcc_novelty.size - rms_novelty.size
    if size_dif > 0:
        rms_novelty = np.pad(rms_novelty, (0, np.abs(size_dif)), mode='edge')
    else:
        mfcc_novelty = np.pad(mfcc_novelty, (0, np.abs(size_dif)), mode='edge')

    novelty = mfcc_novelty * rms_novelty

    peaks_rel_pos, peaks_amp = detect_peaks(novelty)
    """
    save_cmap(mfcc_ssm, 'figures/mfcc_smm.png', ' MFCC Self-Similarity Matrix')
    save_cmap(rms_ssm, 'figures/mfcc_smm.png', ' MFCC Self-Similarity Matrix')
    save_cmap(kernel, 'figures/kernel', 'Checkboard Gaussian Kernel')
    save_line(range(len(novelty)), novelty, 'figures/novelty.png', 'Novelty function', 'Frames', 'Amplitude')
    save_line(peaks_rel_pos, peaks_amp, 'figures/peaks.png', 'Novelty peaks', 'Frames', 'Amplitude', '.')
    """
    peaks_abs_pos = peaks_rel_pos * hop_size

    peak_times = np.zeros_like(song['audio_array'])

    for i in range(len(peaks_abs_pos)):
        beat_peak = find_near_beat(peaks_abs_pos[i], song['beat_times'])
        peak_times[beat_peak] = 1

    return peak_times



def mfcc_structural_similarity_matrix(song, frame_size, hop_size):

    mspec = librosa.feature.melspectrogram(song['audio_array'], sr=song['frame_rate'], n_mels=128, n_fft=frame_size, window="hann", win_length=frame_size, hop_length=hop_size,)

    log_mspec = librosa.power_to_db(mspec, ref=np.max)

    mfcc = librosa.feature.mfcc(S = log_mspec, sr=song['frame_rate'], n_mfcc=13)

    ssm = sklearn.metrics.pairwise.cosine_similarity(mfcc.T, mfcc.T)
    
    ssm -= np.average(ssm)
    m = np.min(ssm)
    M = np.max(ssm)
    ssm -= m
    ssm /= np.abs(m) + M

    return ssm


def rms_structural_similarity_matrix(song, frame_size, hop_size):

    rms_list = []
    for frame in FrameGenerator(essentia.array(song['audio_array']), frameSize = frame_size, hopSize = hop_size):
        rms_list.append(np.average(frame**2))

    ssm = sklearn.metrics.pairwise.pairwise_distances(np.array(rms_list).reshape(-1, 1))

    ssm -= np.average(ssm)
    m = np.min(ssm)
    M = np.max(ssm)
    ssm -= m
    ssm /= np.abs(m) + M

    return ssm


def get_checkboard_kernel(dim):

    gaussian_x = scipy.signal.gaussian(2*dim, std = dim/2.0).reshape((-1,1))
    gaussian_y = scipy.signal.gaussian(2*dim, std = dim/2.0).reshape((1,-1))

    kernel = np.dot(gaussian_x,gaussian_y)

    kernel[:dim,dim:] *= -1
    kernel[dim:,:dim] *= -1

    return kernel
    

def apply_kernel(ssm, kernel):

    kernel_dim = int(kernel.shape[0]/2)
    ssm_dim = ssm.shape[0]

    novelty = np.zeros(ssm_dim)

    ssm_padded = np.pad(ssm, kernel_dim, mode='edge')

    for index in range(ssm_dim):
        frame = ssm_padded[index:index+2*kernel_dim, index:index+2*kernel_dim]
        novelty[index] = np.sum(frame * kernel)
    
    novelty /= np.max(novelty)

    return novelty


def detect_peaks(novelty):

    threshold = np.max(novelty) * 0.025
    
    peakDetection = PeakDetection(interpolate=False, maxPeaks=100, orderBy='amplitude', range=len(novelty), maxPosition=len(novelty), threshold=threshold)
    peaks_pos, peaks_ampl = peakDetection(novelty.astype('single'))
    peaks_ampl = peaks_ampl[np.argsort(peaks_pos)]
    peaks_pos = peaks_pos[np.argsort(peaks_pos)]

    return peaks_pos, peaks_ampl


def find_near_beat(position, beat_times):

    position = int(position)

    i_low = 0
    i_up = 0
    while(position - i_low > 0 and beat_times[position-i_low] == 0):
        i_low += 1
    while(position + i_up < len(beat_times) and beat_times[position+i_up] == 0):
        i_up += 1

    if i_low < i_up:
        return position - i_low
    else:
        return position + i_up



def evaluate(playlist):

    for song in playlist:
        # Evaluating sort of acc in bpm detection
        pass

    # print or store or whatever


In [104]:
# Choosing the first song
# either:
# iteratively choosing next song
# tree search for optimal sequence


circle_of_fifths = {
    "major": ["C", "G", "D", "A", "E", "B", "F#", "Db", "Ab", "Eb", "Bb", "F"],
    "minor": ["A", "E", "B", "F#", "C#", "G#", "D#", "Bb", "F", "C", "G", "D"],
}
scale = ["C", "Db", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]


def get_song_sequence(playlist):

    print("Selecting tracks order...")

    not_in_queue = playlist.copy()

    not_in_queue.sort(key=lambda song: song["estimated_bpm"])

    queue = []

    queue.append(not_in_queue.pop(0))

    while not_in_queue:

        next_song = pick_next_song(queue[-1], not_in_queue)
        queue.append(next_song)
        not_in_queue.remove(next_song)

    return queue


def pick_next_song(current, options):
    """
    Explore several strategies

    Example:
        - Selecting candidate inside a +- bpm bounds
        - Picking the most similar one in key
        (see the paper for inspiration in distances between keys)
    """

    threshold = 4

    selection = None
    current_bpm = current["estimated_bpm"]
    current_key_distance = 12  # Maximum distance

    while not selection:

        for song in options:

            if (
                song["estimated_bpm"] >= current_bpm - threshold
                and song["estimated_bpm"] <= current_bpm + threshold
            ):

                optional_key_distance = key_distance_fifths(
                    current["estimated_key"],
                    current["estimated_mode"],
                    song["estimated_key"],
                    song["estimated_mode"],
                )

                if optional_key_distance < current_key_distance:

                    selection = song
                    current_key_distance = optional_key_distance

        threshold += 2

    return selection


def key_distance_semitones(key1, key2):

    idx1 = scale.index(key1)
    idx2 = scale.index(key2)

    diff = abs(idx1 - idx2)

    distance = min(diff, 12 - diff)

    return distance


def key_distance_fifths(key1, mode1, key2, mode2):

    idx1 = circle_of_fifths[mode1].index(key1)
    idx2 = circle_of_fifths[mode2].index(key2)

    diff = abs(idx1 - idx2)

    distance = min(diff, 12 - diff)

    return distance


In [114]:
# Iteratively:
# Create the transition for one pair of songs
#   - Time wrapping (progressivelly better)
#   - Key changing (explore strategies)
#   - Align all the sequence according to modified beats (try to do it with downbeats)
#   - Volume fades to mix both
import numpy as np
import rubberband as rb


def create_transitions(queue):

    mix = queue[0]

    print("Creating_transitions...")

    for i in range(1, len(queue)):

        print(f"\tMixing tracks {i} and {i+1}...")
        mix = mix_pair(mix, queue[i])

    return mix


def mix_pair(previous_mix, next_song):
    """
    output
        mix = {
        name ([string])
        audio_array (np.array)
        sampling_rate ([double])
        ...
        real_bpm ([Int])
        estimated_bpm ([Int])
        estimated_key ([String])
        cue_points (np.array)
        }
    """

    # selecting the actual cue-points from all the posibilities
    previous_mix_cue_point = select_cue_points(previous_mix)

    print("\t\tAligning songs...")
    next_song_aligned = align(next_song)

    print("\t\tMixing beats...")
    previous_mix_stretched,next_song_stretched,previous_ending,next_beginning = time_wrap(previous_mix, next_song_aligned, previous_mix_cue_point)

    #print("\t\tTransposing keys...")
    # previous_mix, next_song = key_change(previous_mix_stretched, next_song_stretched)

    print("\t\tFading transition...")
    previous_mix_faded, next_song_faded = fade(previous_mix_stretched, next_song_stretched, previous_ending, next_beginning)

    print("\t\tCombining tracks...")
    mix = combine_songs(previous_mix_faded, next_song_faded, previous_ending)

    return mix #, previous_mix_faded, next_song_faded


def select_cue_points(previous_mix):

    max_transition_length = 120

    cue_point = np.zeros_like(previous_mix["cue_points"])

    possible_idx = np.where(previous_mix["cue_points"] == 1)[0]

    # cue_point_idx = possible_idx[
    #    possible_idx
    #    > previous_mix["cue_points"].size
    #    - max_transition_length * previous_mix["frame_rate"]
    # ]
    # print(cue_point_idx)

    # if len(cue_point_idx) > 1:
    #    cue_point_idx = cue_point_idx[1]
    # else:
    #    cue_point_idx = cue_point_idx[0]
    flag = False
    i = 1
    while flag == False:
        # select first cue point that are at least 20s from end.
        if (len(previous_mix["audio_array"]) - possible_idx[-i]) / previous_mix["frame_rate"] >= 20:
            cue_point[possible_idx[-i]] = 1
            flag = True
        i += 1
    #cue_point_idx = possible_idx[-2]
    

    return cue_point


def align(next_song):

    first_beat = np.where(next_song["beat_times"] == 1)[0][0]

    new_next = next_song.copy()

    new_next["audio_array"] = next_song["audio_array"][first_beat:]
    new_next["beat_times"] = next_song["beat_times"][first_beat:]
    new_next["cue_points"] = next_song["cue_points"][first_beat:]

    return new_next


def time_wrap(previous_mix, next_song, previous_mix_cue_point):

    avg_bpm = (previous_mix["estimated_bpm"] + next_song["estimated_bpm"]) / 2

    ending_stretching_ratio = previous_mix["estimated_bpm"] / avg_bpm
    beginning_stretching_ratio = next_song["estimated_bpm"] / avg_bpm

    cue_point_idx = np.where(previous_mix_cue_point == 1)[0][0]

    #NEW-------------
    
    transition_length_seconds = 20

    transition_length_prev_frames_stretched = transition_length_seconds * previous_mix["frame_rate"]
    transition_length_prev_frames = int(transition_length_prev_frames_stretched / ending_stretching_ratio)

    transition_length_next_frames_stretched = transition_length_seconds * next_song["frame_rate"]
    transition_length_next_frames = int(transition_length_next_frames_stretched / beginning_stretching_ratio)
    """
    print('beg len samp: ', transition_length_next_frames)
    print('end len samp: ', transition_length_prev_frames)

    print('beg bpm', previous_mix["estimated_bpm"])
    print('end bpm', next_song["estimated_bpm"])

    print('beg stretch', beginning_stretching_ratio)
    print('end stretch', ending_stretching_ratio)
    """
    ending_audio = previous_mix["audio_array"][cue_point_idx : cue_point_idx + transition_length_prev_frames]
    ending_beats = previous_mix["beat_times"][cue_point_idx : cue_point_idx + transition_length_prev_frames]
    beginning_audio = next_song["audio_array"][:transition_length_next_frames]
    beginning_beats = next_song["beat_times"][:transition_length_next_frames]

    """
    # ending_length_samples = previous_mix["audio_array"].size - cue_point_idx
    ending_length_samples = 20 * previous_mix["frame_rate"]
    transition_length = ending_length_samples * ending_stretching_ratio
    transition_length_seconds = transition_length / previous_mix["frame_rate"]
    # if transition_length_seconds > 20:
    #    transition_length_seconds = 20
    
    print(transition_length_seconds)

    beginning_length_stretched = transition_length_seconds * next_song["frame_rate"]
    beginning_length_samples = int(beginning_length_stretched * beginning_stretching_ratio)

    print('beg len samp: ', beginning_length_samples)
    print('end len samp: ', ending_length_samples)

    ending_audio = previous_mix["audio_array"][cue_point_idx : cue_point_idx + ending_length_samples]
    ending_beats = previous_mix["beat_times"][cue_point_idx : cue_point_idx + ending_length_samples]
    beginning_audio = next_song["audio_array"][:beginning_length_samples]
    beginning_beats = next_song["beat_times"][:beginning_length_samples]

    """

    ending_audio_stretched = rb.stretch(np.array(ending_audio, dtype="int32"),rate=previous_mix["frame_rate"],ratio=ending_stretching_ratio,crispness=6,formants=False,precise=True)
    beginning_audio_stretched = rb.stretch(np.array(beginning_audio, dtype="int32"),rate=next_song["frame_rate"],ratio=beginning_stretching_ratio,crispness=6,formants=False,precise=True)

    """
    print("end: ", len(ending_audio_stretched))
    print("start: ", len(beginning_audio_stretched))
    """
    ending_beats_stretched = stretch_beats(ending_beats, ending_stretching_ratio, ending_audio_stretched.size)
    beginning_beats_stretched = stretch_beats(beginning_beats, beginning_stretching_ratio, beginning_audio_stretched.size)

    previous_mix["estimated_bpm"] = next_song["estimated_bpm"]

    new_previous = previous_mix.copy()
    #new_previous["audio_array"] = np.concatenate((new_previous["audio_array"][:-ending_length_samples], ending_audio_stretched))
    #new_previous["beat_times"] = np.concatenate((new_previous["beat_times"][:-ending_length_samples], ending_beats_stretched))
    #new_previous["cue_points"] = np.concatenate((new_previous["cue_points"][:-ending_length_samples],np.zeros(ending_audio_stretched.size, dtype=previous_mix["cue_points"].dtype)))

    
    new_previous["audio_array"] = np.concatenate((new_previous["audio_array"][:cue_point_idx], ending_audio_stretched))
    new_previous["beat_times"] = np.concatenate((new_previous["beat_times"][:cue_point_idx], ending_beats_stretched))
    new_previous["cue_points"] = np.concatenate((new_previous["cue_points"][:cue_point_idx],np.zeros(ending_audio_stretched.size, dtype=previous_mix["cue_points"].dtype)))

    new_next = next_song.copy()
    new_next["audio_array"] = np.concatenate((beginning_audio_stretched, new_next["audio_array"][transition_length_next_frames:]))
    new_next["beat_times"] = np.concatenate((beginning_beats_stretched, new_next["beat_times"][transition_length_next_frames:]))
    new_next["cue_points"] = np.concatenate((np.zeros(beginning_audio_stretched.size, dtype=next_song["cue_points"].dtype),next_song["cue_points"][transition_length_next_frames:]))

    #return (new_previous,new_next,new_previous["audio_array"][:-ending_length_samples].size,beginning_audio_stretched.size)
    return (new_previous,new_next,new_previous["audio_array"][:cue_point_idx].size,beginning_audio_stretched.size)

def stretch_beats(beat_times, stretching_ratio, desired_length):

    new_beats = []

    zero_sequence_length = 0
    for i in beat_times:
        if i == 0:
            zero_sequence_length += 1
        elif i == 1:
            new_beats += [0] * int(zero_sequence_length * stretching_ratio)
            new_beats += [1]
            zero_sequence_length = 0

    diff = desired_length - len(new_beats)
    if diff > 0:
        new_beats += [0] * diff

    return np.array(new_beats, dtype=int)


def key_change(previous_mix, next_song, previous_mix_cue_point, next_song_cue_point):

    # rubberband

    # Choose to change the key of next_song completely or only the transition part

    return previous_mix, next_song


def fade(previous_mix, next_song, previous_mix_cue_point, next_song_cue_point):

    fade_seconds = 20
    fade_frames = fade_seconds * previous_mix["frame_rate"]

    for i in range(fade_frames):

        #exponential fade
        #previous_mix["audio_array"][-i] = previous_mix["audio_array"][-i] * (1.1 - np.exp(2.398 * (1 - i / fade_frames)) * 0.1)
        #next_song["audio_array"][i] = next_song["audio_array"][i] * (0.1 * np.exp(2.398 * i / fade_frames) - 0.1)

        #linear fade
        previous_mix["audio_array"][-i] = previous_mix["audio_array"][-i] * i/fade_frames
        next_song["audio_array"][i] = next_song["audio_array"][i] * i/fade_frames

    return previous_mix, next_song


def combine_songs(previous_mix, next_song, previous_ending):

    mix = previous_mix.copy()

    next_audio_padded = np.pad(next_song["audio_array"], (previous_ending, 0), constant_values=0)
    next_beat_padded = np.pad(next_song["beat_times"], (previous_ending, 0), constant_values=0)
    next_cue_padded = np.pad(next_song["cue_points"], (previous_ending, 0), constant_values=0)

    mix["audio_array"] = next_audio_padded
    mix["beat_times"] = next_beat_padded
    mix["cue_points"] = next_cue_padded

    mix["audio_array"][: previous_mix["audio_array"].size] += previous_mix["audio_array"]
    mix["beat_times"][: previous_mix["beat_times"].size] += previous_mix["beat_times"]
    mix["cue_points"][: previous_mix["cue_points"].size] += previous_mix["cue_points"]

    return mix


In [9]:
import matplotlib.pyplot as plt

def save_cmap(matrix, filename, title='', xlabel='', ylabel='', colorbar=False):

    fig, ax = plt.subplots()

    c = ax.pcolormesh(matrix, shading='auto', cmap='magma')
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(xlabel)
    if colorbar:
        fig.colorbar(c, ax=ax)

    plt.savefig(filename)


def save_line(x, y, filename, title='', xlabel='', ylabel='', style=''):
    fig, ax = plt.subplots()

    plt.plot(x, y, style)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    plt.savefig(filename)

In [122]:
load_path = "songs/dev_songs_pop2020s/"
store_path = "songs/dev_songs_pop2020s_output/song_mix_new_1.wav"

In [123]:
playlist = load_data(load_path)

Loading data from songs/dev_songs_pop2020s/...
	2 songs loaded


In [124]:
playlist_features = feature_extraction(playlist)

Extracting features
	Song 1 / 2
		Estimating beat...


/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


		Estimating key...


/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


		Estimating cue-points
	Song 2 / 2
		Estimating beat...


/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


		Estimating key...


/usr/local/lib/python3.9/site-packages/madmom/io/audio.py:493: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


		Estimating cue-points


In [125]:
queue = get_song_sequence(playlist_features)
for song in queue:
    print(song['estimated_bpm'], ' ', song['song_name'])

Selecting tracks order...
97.9951841565256   Just Wanna Be Friends
122.3857498889914   Go On Together


In [126]:
mix = create_transitions(queue)

Creating_transitions...
	Mixing tracks 1 and 2...
		Aligning songs...
		Mixing beats...
		Transposing keys...
		Fading transition...
		Combining tracks...


In [127]:
store_song(mix, store_path)
#store_song(previous_mix_faded, "songs/dev_songs_house_output/prev_mix_faded_linear.wav")
#store_song(next_song_faded, "songs/dev_songs_house_output/new_song_faded_linear.wav")